In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
type(df_train)

pandas.core.frame.DataFrame

In [4]:
df_train=pd.concat([df_train,df_valid],axis=0)

In [5]:
df_train.shape

(75000, 4)

In [6]:
df_valid=df_test.copy()

In [7]:
df_train=df_train.loc[df_train['label']<2]

In [8]:
df_train.shape

(25000, 4)

In [9]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [10]:
type(df_train['tokens'])

pandas.core.series.Series

In [11]:
bs=52 # 52 - Jeremey, 20 - default

In [12]:
bptt= 70 #70 - Jeremey, 35 - default

In [13]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [14]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [15]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [16]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [17]:
padlen=1400
padding_idx=1

In [18]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [19]:
df_train['n_tok'].describe()

count    25000.000000
mean       304.945840
std        224.981807
min         16.000000
25%        166.000000
50%        228.000000
75%        371.000000
max       3354.000000
Name: n_tok, dtype: float64

In [20]:
df_train.sort_values(by='n_tok', inplace=True)

In [21]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [22]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [23]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400


In [24]:
df_train.shape

(25000, 5)

In [25]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [26]:
df_train.shape, df_valid.shape

((25000, 5), (25000, 5))

In [27]:
padlen

1400

In [28]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [29]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [30]:
padlen

1400

In [31]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [32]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [33]:
for xb,yb,xlen in dltrain:
    break

In [34]:
xb, yb, xlen

(tensor([[41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         ...,
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1]]),
 tensor([1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
         1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
         0, 0, 0, 1]),
 tensor([409, 470, 182,  61, 427, 170, 426, 727, 281, 452,  75, 172, 421, 197,
         126, 635, 158, 300, 816, 234, 378, 261, 149, 117, 374, 193, 567, 185,
         298, 311,  79, 249, 399, 578, 744, 616, 180, 158, 523, 351, 157, 449,
         313, 407, 321, 542, 149, 156, 169, 379, 141, 243]))

## Model Architecture

In [35]:
from torch import nn
import torch
from torch.autograd import Variable

In [36]:
bs,bptt

(52, 70)

In [37]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=256 #256
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-7
bidirectional=True
dropout_e=0.2 # 0.5
dropout_o=0.5 #0.5
n_out=1

In [38]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [39]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [40]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [41]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [90]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out=n_out
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        model.dropout, model.dropout_o, model.dropout_e = dropout, dropout_o, dropout_e
    
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
             # embs are going to be of shape n_batch * n_seq * n_emb
            # h and c are of shape n_layers * n_batch * n_hidden
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
        
                # output is of shape n_batch * n_seq * n_hidden
            # output [:,-1,:] is the same as hn[-1]
        self.dropout_op=nn.Dropout(self.dropout_o)
        self.fc=nn.Linear(self.n_hidden*2,self.n_out)

        #self.log_softmax=nn.LogSoftmax()
        self.sigmoid=nn.Sigmoid()
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb,Xb_lengths):
        embs=self.dropout_enc(self.encoder(Xb))
        
        #import pdb
        #pdb.set_trace()
        #embs=self.dropout_enc(embs)
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())

        #import pdb
        #pdb.set_trace()
        # before lstm call
            # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)

        # after lstm call
        lstm_out,_=pad_packed_sequence(lstm_out,batch_first=True)
        #lstm_out=self.dropout_op(lstm_out)      
        
        # Wrap the hidden state in a new tensor without the gradients
        #self.hidden=(Variable(newhidden[0].data,requires_grad=False).to(self.device),Variable(newhidden[1].data,requires_grad=False).to(self.device))
        
            # lstm_out has the values that are padded
        #out=self.fc (lstm_out[:,-1,:])
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        
        preds=self.fc(hidden)
        
              
        
        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        
        return preds.view(-1), loss  

In [43]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [44]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [45]:
device="cpu"

## Load pretrained model weights

In [46]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/pretrained_lm_weights','rb'))

In [47]:
if 1==0:
    model=torch.load (f'{PATH}/inter/model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights','wb'))

### Check if model forward works

In [48]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 256, 2, True, 52, 'cpu', 0.5, 0.5, 0.5, 1)

In [49]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (59972, 400)


In [50]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [51]:
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 26,913,857 trainable parameters


In [52]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.5)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (criterion): BCEWithLogitsLoss()
)

In [53]:
model_sentiment.forward(xb,yb,xlen)

(tensor([ 0.0415,  0.0344,  0.0018,  0.0841,  0.0414,  0.0935,  0.0823,  0.0710,
          0.1126,  0.0460, -0.0339,  0.0157,  0.0158, -0.0062, -0.0680,  0.0780,
          0.0629,  0.0065,  0.0757,  0.0475,  0.0257, -0.0033, -0.0053,  0.0406,
          0.0949,  0.0832, -0.0116, -0.0234,  0.0689,  0.0832,  0.0138,  0.0247,
          0.0859,  0.0132,  0.0543,  0.0472,  0.0736,  0.0221,  0.0084, -0.0010,
          0.0088,  0.0376,  0.0127,  0.1119,  0.0722,  0.0162, -0.0004,  0.0046,
          0.0614,  0.0308,  0.0509,  0.0401], grad_fn=<ViewBackward>),
 tensor(0.6945, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [54]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [55]:
preds.view(-1)

tensor([-0.0315,  0.0344,  0.0411,  0.0182,  0.0574,  0.0405,  0.0401,  0.0143,
        -0.0063,  0.0031,  0.0338,  0.0315,  0.0821,  0.0395,  0.0073,  0.0669,
         0.0595,  0.0336,  0.0320, -0.0402, -0.0295,  0.0647,  0.0360, -0.0134,
         0.0715,  0.0451, -0.0064,  0.0409,  0.0881,  0.0748,  0.0438,  0.0564,
         0.0385,  0.0612,  0.0220, -0.0111,  0.0084, -0.0021,  0.0427,  0.1133,
         0.0926,  0.0574,  0.0579,  0.0408,  0.0569, -0.0252,  0.0090, -0.0183,
         0.0078, -0.0104,  0.0629,  0.0153], grad_fn=<ViewBackward>)

In [56]:
yb

tensor([1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
        1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 1])

In [57]:
preds.size()

torch.Size([52])

In [58]:
yb.size()

torch.Size([52])

In [59]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.5000)

In [60]:
from sklearn.metrics import roc_auc_score

In [61]:
roc_auc_score(yb,preds)

0.4562962962962963

In [62]:
pretrained_lm_weights

array([[-9.038397e-02, -1.795423e-02, -3.457930e-02,  8.671192e-02, ..., -3.003510e-02, -1.899482e-02,  1.426260e-02,
        -4.948893e-02],
       [-2.453323e-02, -3.018817e-02,  6.663179e-02, -5.793565e-02, ...,  1.840935e-02,  6.371137e-02,  9.835491e-03,
        -2.131385e-03],
       [ 7.696263e-03, -4.266643e-02, -1.533517e-01,  2.239776e-01, ...,  9.869517e-02,  3.041433e-02,  1.824751e-01,
         1.134978e-01],
       [ 3.305928e-02,  2.266591e-01, -4.264669e-02,  1.490862e-01, ...,  3.407921e-02, -6.422209e-03,  3.180612e-01,
         9.373549e-02],
       ...,
       [ 6.027538e-02, -5.980809e-02,  1.861691e-01, -3.105092e-02, ..., -2.764457e-02,  1.962678e-02, -2.172215e-03,
         6.297247e-02],
       [-1.592789e-02, -2.781571e-04,  1.301994e-01,  2.851282e-02, ...,  5.064877e-02,  1.670864e-01,  2.283701e-02,
        -8.746398e-03],
       [ 4.669700e-02,  3.138980e-02,  1.221957e-02, -3.927753e-02, ..., -1.281436e-01,  1.121320e-01, -3.726090e-03,
        -3.777364e

## Learner

In [63]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [64]:
device="cuda:0"

In [65]:
n_layers

2

In [66]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (59972, 400)


In [67]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_binomial

In [68]:
len(dltrain),len(dlvalid)

(481, 481)

In [69]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [70]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,50,0.25)

In [71]:
model_sentiment.freeze_embedding()

In [72]:
model_sentiment.encoder.weight.requires_grad

False

In [73]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.7049101042747498  0.5215384811162949
Batch:100 0.6640003144741058  0.5871154063940048
Batch:150 0.6459228873252869  0.6179487411181132
Batch:200 0.6539176434278489  0.6095192535221576
Batch:250 0.6653938720226288  0.5893846372365952
Batch:300 0.6683918231725693  0.5835897647341093
Batch:350 0.670021391596113  0.5876373835972377
Batch:400 0.673944260776043  0.5757692511379718
Batch:450 0.6768298112021552  0.5655983105632993
Epoch:0 Loss:0.6769196791361375
Batch:50 0.6005768239498138  0.8353846478462219
Batch:100 0.600014768242836  0.8265384948253631
Batch:150 0.5986782133579254  0.8292308024565379
Batch:200 0.5994284456968307  0.826346186697483
Batch:250 0.6003772809505462  0.8225384938716889
Batch:300 0.6006900773445765  0.8207692623138427
Batch:350 0.6010935039179666  0.8186264043194907
Batch:400 0.6023291030526161  0.8137981072068214
Batch:450 0.6033195376396179  0.8091880640718672
Epoch:0 Loss:0.6769196791361375 Accuracy:0.5664961017714717 Loss:0.6036174146152583 Accuracy

In [74]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.32560369729995725  0.867692346572876
Batch:100 0.3149690940976143  0.8713461899757385
Batch:150 0.31616545885801317  0.8715384972095489
Batch:200 0.31540725149214266  0.8711538809537888
Batch:250 0.3134321436882019  0.8712308046817779
Batch:300 0.3125024070839087  0.8723718305428823
Batch:350 0.30839487544127875  0.8742308049542563
Batch:400 0.307709935195744  0.8755769589543343
Batch:450 0.3049015839563476  0.8768803780608707
Epoch:0 Loss:0.3044661101952908
Batch:50 0.25211650103330613  0.9053846561908722
Batch:100 0.24904165856540203  0.9059615749120712
Batch:150 0.2429587238530318  0.9084615774949392
Batch:200 0.2418766375631094  0.9086538863182068
Batch:250 0.2432983710169792  0.9071538860797882
Batch:300 0.24523801234861214  0.9048077317078909
Batch:350 0.24594865164586477  0.9039560835702078
Batch:400 0.24590014100074767  0.9041346554458142
Batch:450 0.2474136440621482  0.90337610801061
Epoch:0 Loss:0.3044661101952908 Accuracy:0.8770230651645303 Loss:0.2470574180853094

In [75]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.25534069001674653  0.9026923477649689
Batch:100 0.2483185590058565  0.9026923459768296
Batch:150 0.24761820728580156  0.903461578687032
Batch:200 0.25559049379080534  0.9004808086156845
Batch:250 0.25234098213911055  0.9020769622325897
Batch:300 0.254256089652578  0.9015384995937348
Batch:350 0.2554201994410583  0.9002747632775988
Batch:400 0.25505498334765436  0.900625037997961
Batch:450 0.2536403034130732  0.9006410633193122
Epoch:0 Loss:0.25531334081459445
Batch:50 0.22183472752571107  0.9138461887836457
Batch:100 0.22053509801626206  0.9146154218912125
Batch:150 0.2157799701889356  0.9162820887565613
Batch:200 0.21453345000743865  0.9173077312111855
Batch:250 0.21779062893986703  0.9156154232025147
Batch:300 0.22029575732847054  0.9142949110269547
Batch:350 0.2213040920453412  0.9135714680807931
Batch:400 0.22202144099399448  0.9135577318072319
Batch:450 0.22457711691657703  0.9120513210031721
Epoch:0 Loss:0.25531334081459445 Accuracy:0.8999200759459434 Loss:0.2252211251

In [76]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-4

In [77]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.22361971259117128  0.9130769634246826
Batch:100 0.2279772774502635  0.9100000381469726
Batch:150 0.23540648189683755  0.9078205509980519
Batch:200 0.22933210765942932  0.9100000396370888
Batch:250 0.22959483601152897  0.9086154239177704
Batch:300 0.22942450220386187  0.9090385007858276
Batch:350 0.23072864936930793  0.908626412664141
Batch:400 0.23218061966821552  0.9079808077216148
Batch:450 0.2308107333050834  0.9082906368043687
Epoch:0 Loss:0.23166188205787894
Batch:50 0.20899285078048707  0.9211538803577423
Batch:100 0.20655105393379927  0.9219231122732162
Batch:150 0.20229740180075167  0.9221795233090718
Batch:200 0.20147294705733657  0.9223077300190926
Batch:250 0.20385502518713475  0.9214615762233734
Batch:300 0.2055590144669016  0.9197436277071634
Batch:350 0.20601352126470635  0.9195604772227151
Batch:400 0.20609171100892126  0.9193269607424736
Batch:450 0.20773702655401494  0.9180769610404969
Epoch:0 Loss:0.23166188205787894 Accuracy:0.9081961046633255 Loss:0.20758

In [78]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/sentiment_model_awd_lstm_state_dict')
torch.save(optimizer.state_dict(),f'{PATH}/inter/sentiment_learner_awd_lstm_state_dict')
torch.save (model_sentiment,f'{PATH}/inter/sentiment_model_awd_lstm')
torch.save (optimizer,f'{PATH}/inter/sentiment_optimizer_awd_lstm')
torch.save (learner,f'{PATH}/inter/sentiment_learner_awd_lstm')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [79]:
model_sentiment.unfreeze_embedding()

In [80]:
model_sentiment.encoder.weight.requires_grad

True

In [81]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.22543422520160675  0.9038461852073669
Batch:100 0.2261739369481802  0.9050000375509262
Batch:150 0.22177489136656126  0.9084615770975749
Batch:200 0.21774300549179315  0.91182696133852
Batch:250 0.21418449717760085  0.9142308073043823
Batch:300 0.2120815611630678  0.915512859026591
Batch:350 0.21183441687907492  0.9158791598251887
Batch:400 0.21211563305929304  0.9160096538066864
Batch:450 0.2127698947985967  0.915982944700453
Epoch:0 Loss:0.21325347400628603
Batch:50 0.20118361279368402  0.9200000381469726
Batch:100 0.1982692200690508  0.9226923459768295
Batch:150 0.19237979829311372  0.9258974746863047
Batch:200 0.19060995291918517  0.9265385004878044
Batch:250 0.19231294970214366  0.9246923460960388
Batch:300 0.19382860192408163  0.9233974744876225
Batch:350 0.19381565663431372  0.9236813570771899
Batch:400 0.1936470889952034  0.9236058080196381
Batch:450 0.19532251803411377  0.9233761072158814
Epoch:0 Loss:0.21325347400628603 Accuracy:0.9155285851623313 Loss:0.1952587047

In [82]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-5

In [83]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.14470430627465247  0.9461538827419281
Batch:100 0.14433286335319281  0.9482692682743072
Batch:150 0.1429171841343244  0.9467949100335439
Batch:200 0.1421044604294002  0.9464423453807831
Batch:250 0.14251119102537632  0.9461538834571839
Batch:300 0.14152628174672524  0.9466026014089585
Batch:350 0.14243143741573605  0.9463736629486084
Batch:400 0.14166418856941163  0.9473558057844639
Batch:450 0.14269747604098584  0.9475214038954841
Epoch:0 Loss:0.1429498392755549
Batch:50 0.202112774848938  0.9234615743160248
Batch:100 0.20160498678684236  0.9244231134653091
Batch:150 0.19512223991254965  0.9262820871671041
Batch:200 0.19318435352295638  0.9266346520185471
Batch:250 0.1949907910823822  0.9255384986400604
Batch:300 0.19681865897029638  0.9241026016076406
Batch:350 0.1977276628677334  0.9246154223169599
Batch:400 0.19798897867091  0.9239423455297947
Batch:450 0.19988020989629957  0.9234615763028463
Epoch:0 Loss:0.1429498392755549 Accuracy:0.9472693470064667 Loss:0.200225023819

In [88]:
learner.model.dropout, learner.model.dropout_o, learner.model.dropout_e

(0.5, 0.5, 0.5)

In [89]:
learner.model.dropout=0.75, learner.model.dropout_o=0.75, learner.model.dropout_e=0.2

SyntaxError: can't assign to literal (<ipython-input-89-1eb3fb98cfd5>, line 1)